<a href="https://colab.research.google.com/github/yashikart/Blood-Group-Detection-Using-Fingerprints/blob/main/DenseNet121_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import os
import random
import numpy as np
from google.colab import drive
from tensorflow.keras.applications import EfficientNetB3, EfficientNetB0, ResNet50, Xception, ConvNeXtBase, DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from shutil import copyfile, rmtree

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Define dataset paths
original_train_dir = "/content/drive/MyDrive/Blood Group Detection/Gabor filter_1/Train"
test_data_dir = "/content/drive/MyDrive/Blood Group Detection/Gabor filter_1/Test"

# ✅ Temporary directory for balanced dataset
balanced_train_dir = "/content/balanced_train"
if os.path.exists(balanced_train_dir):
    rmtree(balanced_train_dir)
os.makedirs(balanced_train_dir)

# ✅ Ensure 850 images per class
min_train_samples = 850

for class_name in os.listdir(original_train_dir):
    class_path = os.path.join(original_train_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    os.makedirs(os.path.join(balanced_train_dir, class_name), exist_ok=True)

    if len(images) >= min_train_samples:
        selected_images = random.sample(images, min_train_samples)
    else:
        selected_images = images  # Use all available images

    # Copy selected images to balanced directory
    for img in selected_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(balanced_train_dir, class_name, img)
        copyfile(src, dst)

# ✅ Set parameters
img_width, img_height = 224, 224
batch_size = 32
epochs = 20

# ✅ Define Data Generators (No Augmentation)
data_gen = ImageDataGenerator(rescale=1./255)

train_generator = data_gen.flow_from_directory(
    balanced_train_dir, target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical'
)
test_generator = data_gen.flow_from_directory(
    test_data_dir, target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='categorical', shuffle=False
)

num_classes = len(train_generator.class_indices)

# ✅ Check dataset balance
print("\nBalanced Class Distribution:")
for class_name, count in train_generator.class_indices.items():
    print(f"🩸 {class_name}: {min_train_samples} samples")

# ✅ Define Model Architectures
def create_model(base_model_func):
    base_model = base_model_func(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=output)

# ✅ Instantiate models (Including DenseNet121)
models = {
    "DenseNet121": create_model(DenseNet121)
}

# ✅ Train and save models
for model_name, model in models.items():
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    print(f"\n🚀 Training {model_name}...")
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(train_generator, epochs=epochs, validation_data=test_generator, callbacks=[early_stopping])

    # ✅ Save the model
    model_path = f"/content/drive/MyDrive/Blood Group Detection/Models_4/{model_name}.keras"
    model.save(model_path)
    print(f"✅ {model_name} saved to {model_path}")

    # ✅ Print final training accuracy
    train_acc = history.history['accuracy'][-1] * 100
    val_acc = history.history['val_accuracy'][-1] * 100
    print(f"📊 Final Accuracy for {model_name} -> Train: {train_acc:.2f}%, Validation: {val_acc:.2f}%")


Mounted at /content/drive
Found 6800 images belonging to 8 classes.
Found 320 images belonging to 8 classes.

Balanced Class Distribution:
🩸 A+: 850 samples
🩸 A-: 850 samples
🩸 AB+: 850 samples
🩸 AB-: 850 samples
🩸 B+: 850 samples
🩸 B-: 850 samples
🩸 O+: 850 samples
🩸 O-: 850 samples
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

🚀 Training DenseNet121...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 5936s 27s/step - accuracy: 0.4408 - loss: 1.4330 - val_accuracy: 0.5719 - val_loss: 0.9932
Epoch 2/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 5769s 27s/step - accuracy: 0.7068 - loss: 0.7148 - val_accuracy: 0.5906 - val_loss: 0.9059
Epoch 3/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 5773s 27s/step - accuracy: 0.7358 - loss: 0.5903 - val_accuracy: 0.7281 - val_loss: 0.6767
Epoch 4/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 5438s 26s/step - accuracy: 0.7702 - loss: 0.5024 - val_accuracy: 0.7531 - val_loss: 0.5618
Epoch 5/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 5441s 26s/step - accuracy: 0.8040 - loss: 0.4407 - val_accuracy: 0.7469 - val_loss: 0.5833
Epoch 6/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 5457s 26s/step - accuracy: 0.8112 - loss: 0.4146 - val_accuracy: 0.7344 - val_loss: 0.6222
Epoch 7/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 5402s 25s/step - accuracy: 0.8182 - loss: 0.3711 - val_accuracy: 0.7063 - val_loss: 0.7816
Epoch 8/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 5455s 26s/step - accuracy: 0.8294 - loss: 0